In [ ]:
import openai
import langchain
import pinecone 
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import json

In [ ]:
# github = https://github.com/krishnaik06/Complete-Langchain-Tutorials/tree/main/LLM%20Generic%20APP
# youtube = https://www.youtube.com/watch?v=erUfLIi9OFM&t=1809s

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os

In [ ]:
## Lets Read the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [ ]:
doc = read_doc('docs/') ## Page wise docs
print('Loaded Docs: ', len(doc))

In [ ]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [ ]:
documents=chunk_data(docs=doc)
len(documents)

In [ ]:
documents[2:4]

In [ ]:
MISTRAL_API_KEY=os.environ['MISTRAL_API_KEY']
print(MISTRAL_API_KEY)

In [ ]:
## Embedding Technique Of OPENAI
"""
api_key=os.environ['OPENAI_API_KEY']
print(api_key)

pinecone_api_key=os.environ['PINECONE_API_KEY']
print(pinecone_api_key)

embeddings=OpenAIEmbeddings(api_key=api_key)
embeddings

# Initialize a Pinecone client with your API key
from pinecone import Pinecone, ServerlessSpec
import getpass

# if not os.getenv("PINECONE_API_KEY"):
#    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pc = Pinecone(api_key=pinecone_api_key)

import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
index.list

from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
vector_store.add_documents(documents=documents, ids=uuids)

llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")
"""

In [ ]:
from mistralai import Mistral

mistral_model = "mistral-embed"

mistral_client = Mistral(api_key=MISTRAL_API_KEY)

embeddings_batch_response = mistral_client.embeddings.create(
    model=mistral_model,
    inputs=["Embed this sentence.", "As well as this one."],
)

print(embeddings_batch_response)
TOKEN_LENGTH = len(embeddings_batch_response.data[0].embedding)
print('Token Len:', TOKEN_LENGTH)

In [ ]:
from sentence_transformers import SentenceTransformer

st_model = SentenceTransformer("nvidia/NV-Embed-v2", trust_remote_code=True)

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
]
embeddings = st_model.encode(sentences)

print(embeddings)
# [3, 3]

In [ ]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3"
)

In [ ]:
vector = embed.embed_query("How are you?")
print("Vactor len:", len(vector))
print()

In [ ]:
documents[2:4]
print("Len:", len(documents), type(documents[0]))
doc_subset = documents[:10]
print("Len doc_subset:", len(doc_subset))

In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

In [ ]:
data = [
  { 
    "id":uuids[i], 
    "metadata": documents[i].metadata, 
    #"vector": embed.embed_query(documents[i].page_content),
    "vector": mistral_client.embeddings.create( model=mistral_model, inputs=[documents[i].page_content],).data[0].embedding,
    "content": documents[i].page_content 
  } for i in range(len(documents))
]

In [ ]:
print(json.dumps(data[0], indent=4, sort_keys=True))

In [ ]:

print(json.dumps(data[0], indent=4, sort_keys=True))
print(len(data[0]['vector']))
print(len(data[0]['id']))
max_len = 0
for doc in data:
    if len(doc["content"]) > max_len:
        max_len = len(doc["content"])
        
print('max_len:', max_len)

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

In [ ]:
"""
client.create_database(
    db_name="my_database_1"
)
"""

In [ ]:
from pymilvus import DataType

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 2.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.VARCHAR, max_length=36, is_primary=True)
schema.add_field(field_name="content", datatype=DataType.VARCHAR, max_length=(max_len+200))
schema.add_field(field_name="metadata", datatype=DataType.JSON)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=TOKEN_LENGTH)


In [ ]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="vector",
    index_name="vector_index",
    index_type="FLAT",
    metric_type="COSINE",
    params={}
)

In [ ]:
COLLECTION_NAME = "demo_collection"

In [ ]:
if client.has_collection(collection_name=COLLECTION_NAME):
    client.drop_collection(collection_name=COLLECTION_NAME)
    
client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=4096,  # The vectors we will use in this demo has 768 dimensions
    schema=schema,
    index_params=index_params
)

In [ ]:
client.insert(
    collection_name=COLLECTION_NAME,
    data=data
)

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains.question_answering import load_qa_chain

llmModel = OllamaLLM(model="llama3")
chain=load_qa_chain(llmModel, chain_type="stuff")

In [ ]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    query_vector = embed.embed_query(query)
    print(query_vector)
    matching_results = client.search(
        collection_name=COLLECTION_NAME,
        anns_field="vector",
        data=[query_vector],
        limit=k,
        search_params={"metric_type": "COSINE"},
        output_fields=["content", "metadata"]
    )
    return matching_results

In [ ]:
def retrieve_answers(query):
    doc_search=retrieve_query(query, 4)
    from langchain_core.documents import Document

    documents = []
    for hits in doc_search:
        for hit in hits:
            documents.append(Document(page_content=hit["entity"]["content"], metadata=hit["entity"]["metadata"]))
            #print(json.dumps(hit, indent=4, sort_keys=True))
    
    print(json.dumps(doc_search[0], indent=4, sort_keys=True))
            
    ip = {
        "input_documents":documents,
        "question": query
    }
    response=chain.invoke(input=ip)
    return response

In [ ]:
client.load_collection(
    collection_name=COLLECTION_NAME,
    load_fields=["vector", "id", "content", "metadata"] # Load only the specified fields
)

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)


In [ ]:
print("answer: ", type(answer), list(answer.keys()), )
print(answer["output_text"])